### Import stuff

Run this notebook using the 'meanderpy' conda environment.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.signal import savgol_filter
from scipy.spatial import distance
from librosa.sequence import dtw
from tqdm import trange, tqdm
import networkx as nx
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon, Point, MultiLineString, LineString, shape, JOIN_STYLE
from shapely.geometry.polygon import LinearRing
from shapely.ops import snap, unary_union

In [2]:
cd /Users/zoltan/Dropbox/Channels/meanderpy/meanderpy

/Users/zoltan/Dropbox/Channels/meanderpy/meanderpy


In [3]:
import meanderpy as mp

In [20]:
cd /Users/zoltan/Dropbox/meandergraph

/Users/zoltan/Dropbox/Meandergraph


In [22]:
import meandergraph as mg

In [5]:
%matplotlib qt

In [45]:
import sys
print(sys.executable)

/Users/zoltan/anaconda3/envs/meanderpy/bin/python


In [7]:
import geopandas as gpd

In [10]:
from glob import glob #for finding files that match a certain string pattern
lbs = sorted(glob("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb*.shp"))
rbs = sorted(glob("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/rb*.shp"))

In [11]:
X1 = []
X2 = []
Y1 = []
Y2 = []
for fname in lbs:
    lb = gpd.read_file(fname)
    X2.append(lb['geometry'][0].xy[0])
    Y2.append(lb['geometry'][0].xy[1])
for fname in rbs:
    rb = gpd.read_file(fname)
    X1.append(rb['geometry'][0].xy[0])
    Y1.append(rb['geometry'][0].xy[1])

In [74]:
# plot banklines
plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal');

In [38]:
points = plt.ginput(n=2) 
# click twice to select start- and endpoints on first bankline (does not matter which one)

In [14]:
points = [(286850.39156848995, -1660631.8404876508),
 (284610.488612104, -1650178.960024516)]

In [18]:
from scipy import signal, spatial
cl_points = np.vstack((X1[0], Y1[0])).T # coordinates of first centerlines
tree = spatial.KDTree(cl_points)

plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal')
plt.plot(X1[0][tree.query(points[0])[1]], Y1[0][tree.query(points[0])[1]], 
         'ro', zorder=10000)
plt.plot(X1[0][tree.query(points[1])[1]], Y1[0][tree.query(points[1])[1]], 
         'ro', zorder=10000)

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
first_index = tree.query(points[0])[1]
last_index = tree.query(points[1])[1]

first_channel = 0
last_channel = 31

# correlate all lines:    
P1 = []
Q1 = []
for i in trange(len(X1) - 1):
    p, q = mg.correlate_curves(X1[i], X1[i+1], Y1[i], Y1[i+1])
    P1.append(p)
    Q1.append(q)

# correlate all lines:    
P2 = []
Q2 = []
for i in trange(len(X2) - 1):
    p, q = mg.correlate_curves(X2[i], X2[i+1], Y2[i], Y2[i+1])
    P2.append(p)
    Q2.append(q)
    
indices1, x, y = mg.find_indices(first_index, X1, Y1, P1, Q1)
indices2, x, y = mg.find_indices(last_index, X1, Y1, P1, Q1)
for i in range(len(X1)):
    X1[i] = X1[i][indices1[i] : indices2[i]+1]
    Y1[i] = Y1[i][indices1[i] : indices2[i]+1]

indices1, x, y = mg.find_indices(first_index, X2, Y2, P2, Q2)
indices2, x, y = mg.find_indices(last_index, X2, Y2, P2, Q2)
for i in range(len(X2)):
    X2[i] = X2[i][indices1[i] : indices2[i]+1]
    Y2[i] = Y2[i][indices1[i] : indices2[i]+1]

100%|██████████| 30/30 [04:34<00:00,  9.15s/it]


### Resample and correlate centerlines and banklines

In [26]:
# create Z coordinates - just a bunch of zeros -- needed for resampling function in meanderpy:
Z1 = []
for i in range(len(X1)):
    Z1.append(np.zeros(np.shape(X1[i])))
Z2 = []
for i in range(len(X2)):
    Z2.append(np.zeros(np.shape(X2[i])))

# resample centerlines to ds = 2.0 meters:
for i in range(len(X1)):
    x,y,z,dx,dy,dz,ds,s = mp.resample_centerline(X1[i], Y1[i], Z1[i], 2.0)
    X1[i] = x
    Y1[i] = y
for i in range(len(X2)):
    x,y,z,dx,dy,dz,ds,s = mp.resample_centerline(X2[i], Y2[i], Z2[i], 2.0)
    X2[i] = x
    Y2[i] = y

P1, Q1 = mg.correlate_set_of_curves(X1, Y1)
P2, Q2 = mg.correlate_set_of_curves(X2, Y2)

100%|██████████| 30/30 [02:26<00:00,  4.89s/it]


In [47]:
plt.figure()
plt.plot(X1[10], Y1[10])
plt.plot(X2[10], Y2[10])
plt.axis('equal');

In [39]:
from importlib import reload
reload(mg)

<module 'meandergraph' from '/Users/zoltan/Dropbox/Meandergraph/meandergraph/meandergraph.py'>

### Create centerline- and bank graphs

In [51]:
# reload(mg)
ts = len(X1)
# graph = mg.create_graph_from_channel_lines(X[:ts], Y[:ts], P[:ts-1], Q[:ts-1], n_points=20, max_dist=100, remove_cutoff_edges=True)
graph1 = mg.create_graph_from_channel_lines(X1[:ts], Y1[:ts], P1[:ts-1], Q1[:ts-1], n_points=20, max_dist=1000, remove_cutoff_edges=True)
graph2 = mg.create_graph_from_channel_lines(X2[:ts], Y2[:ts], P2[:ts-1], Q2[:ts-1], n_points=20, max_dist=1000, remove_cutoff_edges=True)

100%|██████████| 31/31 [00:00<00:00, 444.14it/s]


In [52]:
# graph = mg.remove_high_density_nodes(graph, min_dist = 10, max_dist = 30)
graph1 = mg.remove_high_density_nodes(graph1, min_dist = 20, max_dist = 60)
graph2 = mg.remove_high_density_nodes(graph2, min_dist = 20, max_dist = 60)

100%|██████████| 31/31 [00:00<00:00, 83.40it/s]


### Plot one of the graphs

In [53]:
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_graph(graph2, ax)
plt.axis('equal');

In [382]:
# save graphs
nx.write_gpickle(graph1, "graph1.gpickle")
nx.write_gpickle(graph2, "graph2.gpickle")

### Create and plot scrolls and bars (connected scrolls)

In [54]:
# reload(mg)
cutoff_area = 1*1e6
fig = plt.figure()
ax = fig.add_subplot(111)
scrolls, scroll_ages, cutoffs, all_bars_graph = mg.create_scrolls_and_find_connected_scrolls(graph1, graph2, cutoff_area, ax)
plt.axis('equal');

100%|██████████| 30/30 [00:00<00:00, 42.40it/s]


### Create and plot 'bar graphs', colored by migration rate

In [55]:
# create bars and bar graphs
min_area = 1000
wbars, poly_graph_1, poly_graph_2 = mg.create_polygon_graphs_and_bar_graphs(graph1, graph2, all_bars_graph, 
                                                                scrolls, scroll_ages, cutoffs, X1, Y1, X2, Y2, min_area, ax)

100%|██████████| 13/13 [00:04<00:00,  2.92it/s]


In [56]:
# plot them, using migration rate
fig = plt.figure(figsize = (12, 12)) 
ax = fig.add_subplot(111)
mg.plot_bar_graphs(graph1, graph2, wbars, ts, cutoffs, 356*24*60*60, X1, Y1, X2, Y2, 300, 1, 0, 500, 'migration', ax)
ax.set_adjustable("box")
ax.axis('equal')
fig.tight_layout()

# ax.plot(X1[ts-1], Y1[ts-1], 'r') # this is needed because Adobe Illustrator does not like the most recent channel polygon
# ax.plot(X2[ts-1], Y2[ts-1], 'r')

100%|██████████| 13/13 [00:03<00:00,  4.00it/s]


In [57]:
# plot one of the polygon graphs
fig = plt.figure()
ax = fig.add_subplot(111)
for node in poly_graph_1.nodes:
    poly = poly_graph_1.nodes[node]['poly']
    ax.add_patch(PolygonPatch(poly, alpha=0.5))
plt.axis('equal');

In [58]:
# add curvature attribute to bankline graphs (needed for curvature maps)
mg.add_curvature_to_line_graph(graph1, smoothing_factor = 51)
mg.add_curvature_to_line_graph(graph2, smoothing_factor = 51)

### Plot migration rate map for one bar

In [61]:
reload(mg)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.add_patch(PolygonPatch(wbars[2].polygon))
mg.plot_migration_rate_map(wbars[2], graph1, graph2, vmin=0, vmax=300, dt=365*24*60*60, saved_ts=1, ax=ax)
plt.axis('equal');

### Plot age map for one bar

In [64]:
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_age_map(wbars[2], vmin=0, vmax=ts, ax=ax)
plt.axis('equal');

### Plot bar polygons and their numbers

In [65]:
# plot bar polygons and their numbers
fig = plt.figure(figsize = (15, 9))
ax = fig.add_subplot(111)
for wbar in wbars:
    ax.add_patch(PolygonPatch(wbar.polygon))
count = 0
for wbar in wbars:
    ax.text(wbar.polygon.centroid.x, wbar.polygon.centroid.y, str(count), fontsize = 16)
    count += 1
plt.axis('equal');

### Some debugging code

In [67]:
# code for debugging bar polygons
from shapely.geometry import Point

def debug_bar_polygons(wbar, graph1, graph2, dt, saved_ts, ax):
    mg.plot_migration_rate_map(wbar, graph1, graph2, vmin=0, vmax=40, dt=dt, saved_ts=saved_ts, ax=ax)
    plt.axis('equal');
    nodes = []
    if wbar.scrolls[-1].bank == 'right':
        line_graph = graph1
    else:
        line_graph = graph2
    for node in line_graph.nodes:
        point = Point(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'])
        if wbar.polygon.contains(point):
            nodes.append(node)
    for node in nodes:
        plt.plot(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'], 'k.')
        plt.text(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'], str(node))

def debug_bar_polygons_2(wbar, graph1, graph2, dt, saved_ts, ax):
    mg.plot_migration_rate_map(wbar, graph1, graph2, vmin=0, vmax=300, dt=dt, saved_ts=saved_ts, ax=ax)
    plt.axis('equal');
    for node in wbar.bar_graph.nodes:
        ax.text(wbar.bar_graph.nodes[node]['poly'].centroid.x, wbar.bar_graph.nodes[node]['poly'].centroid.y, str(node))

In [68]:
dt=365*24*60*60
saved_ts = 1
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_graph(graph2, ax)
fig = debug_bar_polygons_2(wbars[10], graph1, graph2, dt, saved_ts, ax)
# debug_bar_polygons(wbars[10], graph1, graph2, dt, saved_ts, ax)


### Plot radial lines only

In [70]:
fig = plt.figure()
ax = fig.add_subplot(111)
path = mg.find_longitudinal_path(graph2, graph2.graph['start_nodes'][0])
for node in path:
    radial_path, dummy = mg.find_radial_path(graph2, node)
    x = graph2.graph['x'][radial_path]
    y = graph2.graph['y'][radial_path]
    plt.plot(x, y, 'k', linewidth = 0.5)
plt.axis('equal');

### Create and plot 'simple' polygon graph (with only primary radial lines)

In [71]:
graph_poly = mg.create_simple_polygon_graph(graph2, X1[:ts]) # graph2 is left bank
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_simple_polygon_graph(graph_poly, ax, 'left')
plt.axis('equal');

100%|██████████| 403/403 [00:04<00:00, 99.81it/s] 


In [274]:
path = mg.find_longitudinal_path(graph, graph2.graph['start_nodes'][0])

In [72]:
# plot individual polygon 'trajectories'
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(2, len(path), 5):
    radial_path = mg.find_radial_path_2(graph_poly, path[i])
    count = 0
    bank_type = 'left'
    cmap = plt.get_cmap("tab10")
    for node in radial_path:
        if 'poly' in graph_poly.nodes[node].keys():
            if graph_poly.nodes[node]['poly']:
                if bank_type == 'left':
                    if graph_poly.nodes[node]['direction'] == -1:
                        ax.add_patch(PolygonPatch(graph_poly.nodes[node]['poly'], facecolor = cmap(1), edgecolor='k', 
                                                  linewidth = 0.3, alpha = 0.5, zorder = count))
                    if graph_poly.nodes[node]['direction'] == 1:
                        ax.add_patch(PolygonPatch(graph_poly.nodes[node]['poly'], facecolor = cmap(0), edgecolor='k', 
                                                  linewidth = 0.3, alpha = 0.5, zorder = count))
        count += 1
plt.axis('equal');                    